##### import libs

In [1]:
import numpy as np
import pandas as pd
import implicit
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

/home/sokolov/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils import evaluate, load_data

##### read data

In [3]:
user_item_data, user_meta_data, item_meta_data, test_pairs_data = load_data()

In [4]:
user_item_data = user_item_data.merge(
    item_meta_data.drop(columns="embeddings"), on="item_id", how="left"
)

In [5]:
user_item_data["timespent_rel"] = (
    user_item_data["timespent"] / user_item_data["duration"]
)

In [34]:
share_weight = 10
bookmarks_weight = 1
timespent_rel_weight = 50

In [35]:
user_item_data["weighted_target"] = user_item_data["like"]*(
    1 + 
    share_weight*user_item_data.share + 
    bookmarks_weight*user_item_data.bookmarks + 
    timespent_rel_weight*user_item_data.timespent_rel)

##### split data

In [52]:
ui_train, ui_val = train_test_split(user_item_data,
                                    test_size=0.15,
                                    random_state=42,
                                    shuffle=False)

In [53]:
u_train = ui_train.user_id
i_train = ui_train.item_id
likes_train = ui_train.like
dislikes_train = ui_train.dislike

u_val = ui_val.user_id
i_val = ui_val.item_id
likes_val = ui_val.like
dislikes_val = ui_val.dislike

In [54]:
likes_train = ui_train.weighted_target

In [55]:
sparse_train = csr_matrix((likes_train, (u_train, i_train)))
sparse_val = csr_matrix((likes_val, (u_val, i_val)))

##### als model

In [56]:
sparse_train = implicit.nearest_neighbours.tfidf_weight(sparse_train)

In [57]:
model = implicit.als.AlternatingLeastSquares(
    random_state=42,
    )
model.fit(sparse_train, show_progress=True)

/home/sokolov/workspace/vk_recsys/vk_env/lib/python3.10/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.37531518936157227 seconds
  warnings.warn(
100%|██████████| 15/15 [00:43<00:00,  2.92s/it]


##### evaluation

In [42]:
model = model.to_cpu()

In [43]:
als_score = (model.user_factors[ui_val['user_id']] * 
             model.item_factors[ui_val['item_id']]).sum(axis=1)

In [44]:
evaluate(user_id=ui_val.user_id.values,
         target=likes_val.values,
         score=als_score)

100%|██████████| 181721/181721 [00:50<00:00, 3575.84it/s]


0.5929459204150421

In [33]:
np.save("dumps/als_val_score.npy", als_score)

##### submission

обучим модель на всех данных

In [61]:
sparse_train = csr_matrix((user_item_data.weighted_target,
                           (user_item_data.user_id,
                            user_item_data.item_id)))

##### als model

In [62]:
sparse_train = implicit.nearest_neighbours.tfidf_weight(sparse_train)

In [63]:
model = implicit.als.AlternatingLeastSquares(
    random_state=42,
    )
model.fit(sparse_train, show_progress=True)

/home/sokolov/workspace/vk_recsys/vk_env/lib/python3.10/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.49412083625793457 seconds
  warnings.warn(
100%|██████████| 15/15 [00:52<00:00,  3.49s/it]


##### evaluation

In [64]:
model = model.to_cpu()

In [65]:
test_als_score = (model.user_factors[test_pairs_data['user_id']] * 
                  model.item_factors[test_pairs_data['item_id']]).sum(axis=1)

In [66]:
test_pairs_data["predict"] = test_als_score
test_pairs_data.to_csv("./ials_submission.csv",index=False)